# Quran Knowledge Graph Exploration

This notebook demonstrates how to connect to and query the Quran Knowledge Graph using Kuzu.

## Setup

First, let's import the necessary libraries:

In [ ]:
import kuzu
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from tqdm import tqdm

# Set up visualization
plt.style.use('ggplot')
sns.set(style="whitegrid")

print("Libraries imported successfully!")

## Connect to the Quran Knowledge Graph

Now, let's connect to the Kuzu database:

In [ ]:
# Path to your database
# Update this path to where your database is stored
db_path = "../data/quran_graph.kuzu"

try:
    # Connect to the database
    conn = kuzu.Connection(db_path)
    print(f"Successfully connected to the database at {db_path}")
except Exception as e:
    print(f"Error connecting to the database: {e}")
    print("If the database doesn't exist yet, you'll need to create it first.")

## Basic Queries

Let's start with some basic queries to explore the graph:

In [ ]:
# Query to get all surahs (chapters)
try:
    result = conn.execute("""
    MATCH (s:Surah)
    RETURN s.surah_number, s.name_arabic, s.name_english, s.revelation_place, s.verse_count
    ORDER BY s.surah_number
    LIMIT 10
    """)
    
    # Convert to DataFrame for better display
    df = pd.DataFrame(result)
    display(df)
except Exception as e:
    print(f"Error executing query: {e}")

## Query for Specific Verses

In [ ]:
# Query to get verses from Surah Al-Fatiha (Chapter 1)
try:
    result = conn.execute("""
    MATCH (s:Surah {surah_number: 1})-[:CONTAINS]->(v:Verse)
    RETURN v.verse_key, v.text_uthmani, v.text_simple
    ORDER BY v.verse_number
    """)
    
    # Convert to DataFrame
    df = pd.DataFrame(result)
    display(df)
except Exception as e:
    print(f"Error executing query: {e}")

## Explore Word Roots

Let's explore the root words in a specific verse:

In [ ]:
# Query to get words and their roots for a specific verse (e.g., Ayatul Kursi, 2:255)
try:
    result = conn.execute("""
    MATCH (v:Verse {verse_key: '2:255'})-[:HAS_WORD]->(w:Word)-[:HAS_ROOT]->(r:RootWord)
    RETURN w.text_uthmani, w.position, r.text_arabic, r.meaning_english
    ORDER BY w.position
    """)
    
    # Convert to DataFrame
    df = pd.DataFrame(result)
    display(df)
except Exception as e:
    print(f"Error executing query: {e}")

## Thematic Analysis

Let's explore verses related to a specific theme:

In [ ]:
# Query to find verses related to the theme of "patience"
try:
    result = conn.execute("""
    MATCH (t:Topic {name: 'Patience'})<-[r:ADDRESSES_TOPIC]-(v:Verse)
    RETURN v.verse_key, v.text_simple, r.relevance
    ORDER BY r.relevance DESC
    LIMIT 5
    """)
    
    # Convert to DataFrame
    df = pd.DataFrame(result)
    display(df)
except Exception as e:
    print(f"Error executing query: {e}")

## Visualization

Let's create a simple visualization of the relationships between verses and topics:

In [ ]:
# Query to get a small subgraph for visualization
try:
    result = conn.execute("""
    MATCH (v:Verse)-[r:ADDRESSES_TOPIC]->(t:Topic)
    WHERE v.surah_number = 1
    RETURN v.verse_key, t.name, r.relevance
    """)
    
    # Create a graph
    G = nx.Graph()
    
    # Add nodes and edges
    for row in result:
        verse_key = row[0]
        topic_name = row[1]
        relevance = row[2]
        
        G.add_node(verse_key, type='verse')
        G.add_node(topic_name, type='topic')
        G.add_edge(verse_key, topic_name, weight=relevance)
    
    # Create node colors based on type
    node_colors = ['skyblue' if G.nodes[node]['type'] == 'verse' else 'lightgreen' for node in G.nodes]
    
    # Create the plot
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G, seed=42)  # Position nodes using force-directed layout
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=500, alpha=0.8)
    nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)
    nx.draw_networkx_labels(G, pos, font_size=10)
    
    plt.title('Relationships between Verses and Topics')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Error creating visualization: {e}")

## Vector Similarity Search

If your graph includes vector embeddings, you can perform semantic similarity searches:

In [ ]:
# This is a placeholder for vector similarity search
# You'll need to have vector embeddings in your database for this to work

# Example query (commented out as it depends on your specific schema)
'''
try:
    # Define a query embedding (this would typically come from encoding a text query)
    # For demonstration, we'll use a random vector
    import numpy as np
    query_embedding = np.random.rand(768).tolist()  # Assuming 768-dimensional embeddings
    
    # Execute vector similarity search
    result = conn.execute("""
    MATCH (v:Verse)
    WHERE v.embedding VECTOR_DISTANCE($query_embedding) < 0.5
    RETURN v.verse_key, v.text_simple, v.embedding VECTOR_DISTANCE($query_embedding) AS similarity
    ORDER BY similarity
    LIMIT 5
    """, {"query_embedding": query_embedding})
    
    # Convert to DataFrame
    df = pd.DataFrame(result)
    display(df)
except Exception as e:
    print(f"Error executing vector search: {e}")
'''

## Conclusion

This notebook demonstrates basic interactions with the Quran Knowledge Graph using Kuzu. You can extend these examples to perform more complex queries and analyses based on your specific needs.